 # Opis notatnika
 Po pobraniu danych z zewnętrznego serwisu, a następnie załadowaniu ich do naszej wewnętrznej (prawie firmowej) bazy, czas na ich analizę oraz eksplorację.
 Ponieważ zakładamy, że pracujemy z tym zbiorem pierwszy raz, warto przejrzeć go pod dość szerokim kątem, aby nabrać świadomości, jakie informacje są tam ukryte i co stanowi potencjalną wartość biznesową.

 Eksplorację zaczniemy od centralnej bazy danych `flight`, w której nastawimy się w szczególności na zmienną `dep_delay` (za dokumentacją u [źródła](https://www.kaggle.com/datasets/threnjen/2019-airline-delays-and-cancellations?resource=download&select=raw_data_documentation.txt)), która informuje o wysokości opóźnienia odlotu samolotu.
 Wykonując kolejne kroki, najpierw odpowiednio przygotujemy nasz wyjściowy zbiór do analizy, by później zacząć go wzbogacać o dodatkowe informacje, np. pogodowe.

 Dzięki wyciągnięciu wniosków z danych, które otrzymaliśmy, będziemy mogli zaproponować system raportowania wspomagający biznes, czy zdefiniować dalsze kroki, które usprawnią działania lotnisk.

 Powodzenia!

 > Ze względu na objętość zadań w tym obszarze, ten krok podzielony został na kilka mniejszych części.

 # Konfiguracja
 Uzupełnij implementajcę procedury `load_table_from_db`, która będzie odpowiedzialna za
 pobieranie danych z bazy danych oraz zwrócenie ramki do dalszej pracy.

 W trakcie pracy nad jej implementacją możesz wspomóc się następującymi materiałami:
 - `read_sql` - dokumentacja techniczna metody: [klik](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html),
 - `SQL - analiza danych -> Zjazd 1 - materiał dodatkowe -> Export danych -> Python`

 > **Uwaga:**  
 > Metoda powinna tylko pobierać dane z bazy, nie implementuj tutaj dodatkowej logiki.

 Tutaj zaimportuj wymagane biblioteki

In [ ]:
# %pip install matplotlib --quiet
# %pip install dask[dataframe] --quiet

In [ ]:
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import pandas as pd
import time
import dask.dataframe as dd
import multiprocessing
import sys
sys.path.append('../')

pd.set_option("display.max_colwidth", 3000)
pd.set_option("display.max_columns", None)
num_cores = multiprocessing.cpu_count()

%matplotlib inline
print(f"{num_cores=}")

 ## Połączenie z bazą danych
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
from config.DB_connect import username, passwd, hostname, db_name

 Tutaj stwórz zmienną engine, która zostanie użyta do połączenia z bazą danych

In [ ]:
url = f"postgresql://{username}:{passwd}@{hostname}:{port}/{db_name}"
engine = create_engine(url, echo=False)

 Tutaj uzupełnij implementację metody `read_sql_table`

In [ ]:
def read_sql_table(table_name):
    return pd.read_sql(f"SELECT * FROM {table_name}", engine)

 # Wczytanie danych do obszaru roboczego
 Używając metody `read_sql_table`, wczytaj do obszaru roboczego zawartośc tabeli `flight` i zapisz w ramce o nazwie `flight_df_raw`.

 Następnie wykonaj poniższe polecenia:
 1. Usuń z ramki loty, które:
     * odbyły się w 2020 roku,
     * zostały anulowane.
 1. Zmień nazwę kolumny `dep_delay_new` na `dep_delay`.
 1. Tak powstałą tabelę zapisz do ramki, która nazywać się będzie `flight_df` - z tej ramki będziemy korzystali do końca analizy.
 1. Wyznacz, ile kolumn ma tabela `flight_df`, wynik zapisz do zmiennej `flight_df_columns_amount`.
 1. Wyznacz, ile wierszy ma tabela `flight_df`, wynik zapisz do zmiennej `flight_df_rows_amount`.

 Tutaj wczytaj ramkę do obszaru roboczego

In [ ]:
flight_df_raw = read_sql_table("flight")

 Tutaj oczyść ramkę usuwając loty z roku 2020 oraz te anulowane

In [ ]:
flight_df = flight_df_raw.loc[
    (flight_df_raw["year"] != 2020) & (flight_df_raw["cancelled"] == False)
]

 Tutaj zmień nazwę kolumny `dep_delay_new` na `dep_delay`

In [ ]:
flight_df = flight_df.rename(columns={"dep_delay_new": "dep_delay"})

 Tutaj zainicjuj zmienne `flight_df_columns_amount` oraz `flight_df_rows_amount`, które zostaną użyte do sprawdzenia poprawności wykonania tej części

In [ ]:
flight_df_columns_amount = flight_df.shape[1]
flight_df_rows_amount = flight_df.shape[0]

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

 ### Sprawdzenie liczby kolumn

In [ ]:
flight_df_expected_columns_amount = 28
assert (
    flight_df_columns_amount == flight_df_expected_columns_amount
), f"Oczekiwano {flight_df_expected_columns_amount} kolumn, otrzymano {flight_df_columns_amount}"

 ### Sprawdzenie liczby wierszy

In [ ]:
flight_df_expected_rows_amount = 7287112
assert (
    flight_df_rows_amount == flight_df_expected_rows_amount
), f"Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}"

 ### Sprawdzenie czy nie zostały w ramce loty z 2020

In [ ]:
flight_df_year_test = flight_df.loc[flight_df["year"] == 2020].shape[0]
assert (
    flight_df_year_test == 0
), "W ramce `flight_df` nadal znajdują się loty z 2020 roku"

 ### Sprawdzenie czy nie zostały w ramce loty anulowane

In [ ]:
flight_df_cancelled_test = flight_df.loc[flight_df["cancelled"] != 0].shape[0]
assert (
    flight_df_cancelled_test == 0
), "W ramce `flight_df` nadal znajdują się anulowane loty"

 ### Sprawdzenie czy nazwa kolumny została poprawnie zmieniona

In [ ]:
assert (
    "dep_delay" in flight_df.columns
), "Kolumna dep_delay nie została znaleziona w ramce flight_df"

 # Analiza kolumny `dep_delay` cz. 1
 Wyznacz statystyki opisowe dla zmiennej `dep_delay` i zapisz do zmiennej `dep_delay_statistics_df`.
 W ramce powinny znaleźć się następujące wiersze:
 - średnia,
 - mediana,
 - odchylenie standardowe,
 - min, max
 - percentyle `[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`

Wyniki zaokrągl do dwóch miejsc po przecinku.

W trakcie rozwiązywania tego zadania możesz posłużyć się następującymi materiałami:
 - `LMS -> Python-Analiza danych -> Przygotowanie do zjazd 3 -> Podstawy statystyki opisowej`
 - `describe` - dokumentacja techniczna metody: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)

 Tutaj wyznacz statystyki opisowe dla kolumny `dep_delay` oraz zainicjuj ramkę `dep_delay_statistics_df`

In [ ]:
dep_delay_statistics_df = (
    flight_df["dep_delay"].describe(
        [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).round(2)
)

 ## Sprawdzenie wyników
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
expected = {
    "count": 7287112.0,
    "mean": 14.07,
    "std": 47.82,
    "min": 0.0,
    "10%": 0.0,
    "25%": 0.0,
    "50%": 0.0,
    "75%": 7.0,
    "90%": 39.0,
    "95%": 77.0,
    "99%": 200.0,
    "max": 2710.0,
}
dep_delay_statistics_dict = dep_delay_statistics_df.to_dict()

assert (
    dep_delay_statistics_dict == expected
), f"Błąd. Otrzymano wartości : {dep_delay_statistics_dict}"

 # Analiza kolumny `dep_delay` cz. 2
 Przeanalizuj dokładniej kolumnę `dep_delay` wykonując poniższe polecenia:
 1. Wyznacz wykres dla _całej kolumny_ (tzn. tak jak jest).
 1. Wyznacz wykres z pominięciem tych wierszy, dla których `dep_delay=0`.
 1. Obcinając wykres do percentyla 95% oraz pomijając `dep_delay=0`.

Dla wszystkich wykresów użyj histogramu z koszykami co 10 tj. `[0, 10)`, `[10, 20)` i tak dalej.

Możesz tutaj użyć swojego ulubionego narzędzia do tworzenia wykresów - `matplotlib` czy `dash`. Pamiętaj o odpowiednim wystylowaniu każdego z wykresów zgodnie z dobrymi praktykami.

 W trakcie pracy możesz posłużyć się następującymi artykułami:
 - Dla `Matplotlib`:
     - `Python - analiza danych -> Dzień 7 - Wykresy -> Zaawansowane wykresy`
     - `hist` - dokumentacja metody: [klik](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)
 - Dla `Plotly`:
     - `Wizualizacja danych -> Dzień 2 -> Wprowadznie do plotly`
     - `histogram` - dokumentacja metody: [klik](https://plotly.com/python/histograms/)

 Tutaj stwórz wykres dla całej kolumny `dep_delay`

In [ ]:
plt.figure(figsize=(15, 10))
plt.hist(
    flight_df["dep_delay"],
    bins=range(
        int(flight_df["dep_delay"].min()), int(
            flight_df["dep_delay"].max()), 10
    ),
    edgecolor="white",
    align="left",
    log=True,
    label="Ilość odlotów",
)

plt.xticks = range(
    int(flight_df["dep_delay"].min()), int(flight_df["dep_delay"].max()), 10
)

plt.ylabel("Ilość odlotów (skala logarytmiczna)")
plt.xlabel("Czas opóżnienia [min]]")
plt.title(
    "Rozkład wszystkich odlotów od 0 minut opóżnienia, w przedziałach co 10 minut.\nRok 2019"
)
plt.legend()
plt.grid(axis="both")

plt.show()

 Tutaj stwórz wykres dla `dep_delay` używając warunku `dep_delay > 0`

In [ ]:
flight_delays_df = flight_df.loc[(flight_df["dep_delay"] > 0)]

plt.figure(figsize=(15, 10))

plt.hist(
    flight_delays_df["dep_delay"],
    bins=range(
        int(flight_delays_df["dep_delay"].min()),
        int(flight_delays_df["dep_delay"].max()),
        10,
    ),
    edgecolor="white",
    # align='left',
    log=True,
    label="Ilość odlotów",
)

plt.xticks = range(
    int(flight_delays_df["dep_delay"].min()),
    int(flight_delays_df["dep_delay"].max()),
    10,
)

plt.ylabel("Ilość odlotów opóżnionych (skala logarytmiczna)")
plt.xlabel("Czas opóżnienia [min]]")
plt.title(
    "Rozkład wszystkich opóżnionych odlotów w przedziałach opóżnień co 10 min\n Rok 1999"
)
plt.legend()
plt.grid(axis="both")

plt.show()

 Tutaj stwórz wykres dla `dep_delay` używając warunków `dep_delay > 0` oraz `dep_delay < percentile 95%`

In [ ]:
fight_delay_quantile = flight_df[
    (flight_df["dep_delay"] > 0)
    & (flight_df["dep_delay"] < flight_df["dep_delay"].quantile(0.95))
]

plt.figure(figsize=(15, 10))

plt.hist(
    fight_delay_quantile["dep_delay"],
    bins=range(
        int(fight_delay_quantile["dep_delay"].min()),
        int(fight_delay_quantile["dep_delay"].max()),
        10,
    ),
    edgecolor="white",
    rwidth=0.8,
    log=False,
    label="Ilość odlotów",
)

plt.xticks = range(
    int(fight_delay_quantile["dep_delay"].min()),
    int(fight_delay_quantile["dep_delay"].max()),
    10,
)

plt.yticks = range(0, 1000000, 1000)

plt.ylabel("Ilość odlotów opóżnionych (w milionach)")
plt.xlabel("Czas opóżnienia w przedziałach co 10min]")
plt.title("Rozkład opóżnionych odlotów bez 5% największych opóżnień")
plt.legend()
plt.grid(axis="both")

plt.show()

 # Analiza opóźnień
 Zdefiniuj w ramce `flight_df` nową kolumnę - `is_delayed` jako te opóźnienia, które wynosiły więcej niż `(>)` 15 minut.

 Zgodnie z powyższą definicją, wyznacz jaki procent wszystkich lotów był opóźniony. Wynik zapisz do zmiennej `delayed_ratio` z dokładnością do dwóch miejsc po przecinku. Postaraj się, aby wartość tej zmiennej nie była zapisana ręcznie.

 Tutaj stwórz nową kolumnę `is_delayed` oraz odpowiednio ją uzupełnij.

In [ ]:
flight_df["is_delayed"] = flight_df["dep_delay"] > 15

 Tutaj zdefiniuj oraz wyznacz wartość dla zmiennej `delayed_ratio`

In [ ]:
delayed_ratio = (flight_df["is_delayed"].sum() /
                 flight_df["is_delayed"].count()).round(2)

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
delayed_ratio_expected = 0.18
assert (
    delayed_ratio == delayed_ratio_expected
), f"Oczekiwanio {delayed_ratio_expected}, otrzymano {delayed_ratio}"

 # Opóźnienia vs. miesiąc kalendarzowy
 Zbadaj, jak zmienia się odsetek opóźnień w zależności od **miesiąca kalendarzowego**. Zadanie wykonaj w dwóch krokach:
 1. stwórz zmienną `flight_delays_by_month_df` używając metody `groupby`,
 1. na podstawie zmiennej `flight_delays_by_month_df`, wygeneruj odpowiedni wykres zgodnie z dobrymi praktykami.

W trakcie pracy nad tym zadaniem możesz posłużyć się następującymi materiałami z `LMS`:
 - `Python - analiza danych -> Dzień 5 - Pandas -> Grupowanie`
 - `groupby`- dokumentacja metody `Pandas`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

 Tutaj pogrupuj dane, a wyniki zapisz do ramki `flight_delays_by_month_df`

In [ ]:
dask_df = dd.from_pandas(flight_df, npartitions=num_cores)

start_time = time.time()
flight_delays_by_month_df = dask_df.groupby(
    ["month"]).sum()["is_delayed"].compute()

flight_delays_by_month_df.sort_index(inplace=True)
end_time = time.time()

print(f"Czas wykonania: {end_time - start_time} sekund")

 Tutaj narysuj wykres, używając danych z ramki `flight_delays_by_month_df`

In [ ]:
# Przygotowanie danych do wykresu
fig = plt.figure(figsize=(15, 10))
plt.plot(
    flight_delays_by_month_df.index,
    flight_delays_by_month_df,
    label="opóźnione",
    drawstyle="steps-mid",
)
plt.ylabel("Ilość odlotów opóźnionych")
plt.xlabel("Miesiące")
plt.title("Rozkład opóźnionych odlotów wg miesięcy")
plt.legend()
plt.grid(axis="both")
plt.show()

 # Opóźnienia vs. dzień tygodnia cz. 1
 Zbadaj, jak zmienia się odsetek opóźnień w zależności od **dnia tygodnia**. Zadanie wykonaj w dwóch krokach:
 1. stwórz zmienną `flight_delays_by_weekday_df` używając metody `groupby`,
 1. na podstawie zmiennej `flight_delays_by_weekday_df`, wygeneruj odpowiedni wykres zgodnie z dobrymi praktykami.

 Tutaj pogrupuj dane

In [ ]:
start_time = time.time()
flight_delays_by_weekday_df = dask_df.groupby(
    ["day_of_week"]).sum()["is_delayed"].compute()

flight_delays_by_weekday_df.sort_index(inplace=True)
end_time = time.time()

print(f"Czas wykonania: {end_time - start_time} sekund")

 Tutaj narysuj wykres

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(
    flight_delays_by_weekday_df.index,
    flight_delays_by_weekday_df,
    drawstyle="steps-mid",
)

plt.ylabel("Suma opóżnionych wylotów > 15min")
plt.xlabel("Dni tygodnia")
plt.title("Opóżnione wyloty wg dni tygodnia")
plt.grid(axis="both")

plt.show()
plt.close()

 # Opóźnienia vs. dzień tygodnia cz. 2
 Dokonaj agregacji kolumny `day_of_week` do nowej kolumny `is_weekend` w `flight_df`. Jako weekend przyjmij wartości 6, 7.
 1. Używając grupowania, wyznacz odsetek opóźnień w zależności od tego, czy lot odbywał się w weekend czy nie. Wyniki zapisz do ramki `flight_delays_by_weekend_df` oraz zaokrąglij do dwóch miejsc po przecinku.
 1. Zaprezentuj graficznie wynik analizy.
 1. Czy Twoim zdaniem odsetek opóźnień jest zależny od tego, czy lot odbywał się w weekend? Uzasadnij.

 Tutaj dodaj nową kolumnę `is_weekend` do `flight_df`

In [ ]:
flight_df["is_weekend"] = flight_df["day_of_week"] > 5

 Tutaj dokonaj agregacji danych do ramki `flight_delays_by_weekend_df`

In [ ]:
# Assuming flight_df is a Pandas DataFrame
flight_ddf = dd.from_pandas(flight_df, npartitions=num_cores)

# Step 1: Grouping and Summarizing Data
grouped_ddf = flight_ddf.groupby(["is_weekend"]).sum()

# Step 2: Selecting Specific Columns
is_delayed_sum_ddf = grouped_ddf[["is_delayed"]]

# Step 3: Calculating Proportion of Non-Delayed Flights
proportion_non_delayed = flight_ddf["is_delayed"].count(
) - flight_ddf["is_delayed"].sum().compute()

# Step 4: Calculating Final Result
flight_delays_by_weekend_ddf = (
    is_delayed_sum_ddf / proportion_non_delayed).round(2).compute()

# Print or use flight_delays_by_weekend_ddf as needed
print(flight_delays_by_weekend_ddf)

In [ ]:
flight_delays_by_weekend_df = flight_delays_by_weekend_ddf.is_delayed

In [ ]:
flight_delays_by_weekend_df.to_dict()

In [ ]:
# flight_delays_by_weekend_df = (
#     flight_df.groupby(["is_weekend"], as_index=None).sum()[["is_delayed"]]
#     / (flight_df["is_delayed"].count() - flight_df["is_delayed"].sum())
# ).round(2)

 Tutaj narysuj wykres, używając danych z ramki `flight_delays_by_weekend_df`

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(["dni robocze", "weekend"],
         flight_delays_by_weekend_df * 100,
         drawstyle="steps-mid",
         )

plt.xlabel("Dni tygodnia")
plt.ylabel("Udział opóżnionych wylotów > 15min w % do wylotów o czasie")
plt.title("Opóżnione wyloty wg dni tygodnia ")
plt.grid(axis="both")

plt.show()

 ### Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
expected_flight_df_by_weekend = {0: 0.16, 1: 0.06}
assert (
    flight_delays_by_weekend_df.to_dict() == expected_flight_df_by_weekend
), f"Spodziewano się wyników: {expected_flight_df_by_weekend}\n otrzymano  {flight_delays_by_weekend_df}"

 # Opóźnienia vs. odległość lotu
 Przeanalizuj kolumnę `distance`, wykonując poniższe polecenia:
 1. Podobnie jak dla zmiennej `dep_delay`, wyznacz statystyki opisowe oraz dodatkowo przedstaw percentyle `[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]`. Wynik zapisz do zmiennej `flight_distance_analysis_df` oraz zaokrąglij do dwóch miejsc po przecinku.
 1. Nakreśl wykres punktowy (`scatter`) używając `distance` oraz `dep_delay`. Narysuj wykres dla losowych 10 tysięcy wierszy. Czy na takim wykresie możesz coś zaobserwować?
 1. Usuń z ramki `flight_df`, te wiersze, dla których `distance` jest powyżej 95% percentyla.
 1. Używając ramki `flight_df`, dokonaj agregacji zmiennej `distance` co 100 mil do nowej kolumny `distance_agg` oraz wyznacz odsetek opóźnień w każdym koszyku. Wynik zapisz do ramki o nazwie `flight_delays_by_distance_agg_df`.
 1. Narysuj wykres słupkowy, używając danych zapisanych w `flight_delays_by_distance_agg_df`.
 1. Czy Twoim zdaniem większy dystans oznacza większe prawdopodobieństwo opóźnienia lotu? Uzasadnij.

 Wskazówka:
 - Przy generowaniu losowych wierszy przyda sie link do dokumentacji metody `sample`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html)
 - Przy generowaniu grup przyda się link do dokumentacji metody `cut`: [klik](https://pandas.pydata.org/docs/reference/api/pandas.cut.html)

 > Dla dużych zbiorów danych kreślenie wszystkich danych mija się z celem ze względu na czytelność. Mimo że zaprezentujemy pewną część zbioru, zakładamy, że danych na wykresie jest na tyle dużo, że stanowią one reprezentacyjną próbkę populacji.

 Tutaj dokonaj agregacji danych do ramki `flight_distance_analysis_df`

In [ ]:
flight_distance_analysis_df = (
    flight_df["distance"].describe(
        [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99]).round(2)
)

 Tutaj narysuj wykres dla 10 000 losowych wierszy z tabeli `flight_df`

In [ ]:
flight_sample_df = flight_df.sample(10000, replace=False, ignore_index=True)

In [ ]:
plt.figure(figsize=(15, 10))
plt.scatter(flight_sample_df["distance"],
            flight_sample_df["dep_delay"], alpha=0.3)

plt.ylabel("Opóżnienie odlotów w min")
plt.xlabel("Odległość do portu przeznaczenia w milach")
plt.title("Zależność opóżnienia od odległości do celu")
plt.grid(axis="both")
plt.show()

 Tutaj usuń z ramki `flight_df` wiersze, dla których `distance` jest powyżej `95% percentyla` (> 95%)

In [ ]:
flight_df = flight_df[(flight_df["distance"] <=
                       flight_df["distance"].quantile(0.95))]

### Sprawdzenie

In [ ]:
flight_df_expected_rows_amount = 6922924
flight_df_rows_amount = flight_df.shape[0]
assert (
    flight_df_rows_amount == flight_df_expected_rows_amount
), f"Oczekiwano {flight_df_expected_rows_amount} wierszy, otrzymano {flight_df_rows_amount}"

 Tutaj dokonaj agregacji zmiennej `distance` oraz wyznacz odsetek opóźnień

In [ ]:
fight_distance_max = flight_df.distance.max().round(-2)
fight_distance_min = flight_df.distance.min().round(-2)
print(f"maksymalna długość lotu: {flight_df.distance.max()}")
print(f"minimalna długość lotu: {flight_df.distance.min()}")
print(
    f"Maksymalna długość po zaokrągleniu w góre do wielokrotności 100: {fight_distance_max}"
)
print(
    f"Minimalna długość po zaokrągleniu w dół do wielokrotności 100: {fight_distance_min}"
)

In [ ]:
bins_len = 100
flight_df["distance_agg"] = pd.cut(
    flight_df.distance,
    bins=range(
        fight_distance_min,
        fight_distance_max
        + bins_len,  # dodajemy jeden koszyk z prawej strony by zmieścić wartość maksymalną
        bins_len,
    ),
)
# zakresy odleŋłości przypisane do distance
flight_df[["distance", "distance_agg"]]

In [ ]:
flight_delays_sum_count = flight_df.groupby("distance_agg", as_index=None).agg(
    {"is_delayed": ["sum", "count"]}
)
# obliczanie factor jako współczynnika opóżnionych odlotów do wszystkich
flight_delays_sum_count["factor"] = (
    flight_delays_sum_count.is_delayed["sum"] /
    flight_delays_sum_count.is_delayed["count"]
).round(5)

flight_delays_sum_count

In [ ]:
flight_delays_by_distance_agg_df = flight_delays_sum_count[
    ["distance_agg", "factor"]
].copy()
flight_delays_by_distance_agg_df

 Tutaj narysuj wykres słupkowy, używając danych zapisanych w `flight_delays_by_distance_agg_df`

In [ ]:
plt.figure(figsize=(15, 10))
x, y = [], []

for index, row in flight_delays_by_distance_agg_df.iterrows():
    x.append(row["distance_agg"].sort_index().to_string())
    y.append(row["factor"] * 100)

plt.plot(
    x,  # flight_delays_sum_count.index
    y,
    drawstyle="steps-mid",
    label=("Procent opóżnionych odlotów"),
)


plt.xlabel("Dystans")
plt.ylabel("Udział opóżnionych wylotów > 15min w %")
plt.title("Odloty wg grup co 100 mil do lotniska docelowego")
plt.legend()
plt.grid(axis="both")

plt.show()

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana

In [ ]:
assert (
    "distance_agg" in flight_df.columns
), "Nie odnaleziono kolumny distance_agg w ramce flight_df"

 # Opóźnienia vs. grupa odległości
 Przeanalizuj kolumnę `distance_group` dostępą w zbiorze danych oraz odpowiedz na poniższe:
 1. Dla jakich odcinków zostały wyznaczone poszczególne grupy? Wyznacz maksymalną oraz minimalną `distance` wartość w poszczególnych grupach. Wynik zapisz do ramki `flight_distance_by_distance_group`.
 1. Wyznacz prawdopodobieństwo opóźnienia przy użyciu tych grup. Wynik zapisz do ramki `flight_delays_by_distance_group_df`.
 1. Używając ramki `flight_delays_by_distance_group_df`, wykreśl odpowiedni wykres wizualizujący dane.
 1. Na ile zbieżne są wyniki tej analizy z tą wykonaną w poprzednim punkcie?

Wskazówka do punktu pierwszego:
 - Do agregacji danych przyda się metoda `agg`: `Python - analiza danych -> Dzień 5 - Pandas -> Grupowanie`
 - Dokumentacja metody `agg`: [klik](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html)

Tutaj wyznacz odcinki, dla których zostały wyznaczone poszczególne grupy

In [ ]:
flight_df.groupby("distance_group").agg({"distance": ["min", "max"]})

 Tutaj wyznacz odsetek opóźnień w każdej grupie zapisując wyniki do ramki `flight_delays_by_distance_group_df`

In [ ]:
df_tmp = flight_df.groupby("distance_group", as_index=True).agg(
    {"is_delayed": ["sum", "count"]}
)
df_tmp["factor"] = (
    df_tmp.is_delayed["sum"]
    / (
        df_tmp.is_delayed["count"]
        # - df_tmp.is_delayed["sum"]
    )
).round(4)
df_tmp

In [ ]:
flight_delays_by_distance_group_df = df_tmp[["factor"]].copy()
flight_delays_by_distance_group_df

 Tutaj narysuj wykres przy użyciu ramki `flight_delays_by_distance_group_df`

In [ ]:
plt.figure(figsize=(15, 10))

plt.plot(
    flight_delays_by_distance_group_df.index,
    flight_delays_by_distance_group_df["factor"] * 100,
    drawstyle="steps-mid",
)

plt.xlabel("Grupa Odległości")
# plt.xticks(flight_delays_by_distance_agg_df['distance_agg'].to_string())
plt.ylabel("Udział opóżnionych wylotów > 15min w %")
plt.title("Opóżnione wyloty wg grup dystansu")

plt.grid(axis="both")
plt.show()

 ## Czy większy dystans oznacza większe prawdopodobieństwo opóźeninia lotu?
 Miejsce na Twój komentarz - czy wykresy można porównać? Czy dają takie same wnioski?

Wykresy można porównać. Rozkład opóżnienień zagregowanych co 100 mil jest rosnący wraz z dystansem, tak jak rozkład wg grup co 250mil.
Wraz z odległością planowanych odlotów rośnie spodziewane opóżnienie

 # Podsumowanie
 W tym notatniku dość dokładnie przeanalizowaliśmy ramkę `fligh_delays` bez wzbogacania jej o dodatkowe dane z innych źródeł, takich jak dane pogodowe.

 Zanim przejdziemy dalej, należy zapisać bieżącą postać ramki (najlepiej lokalnie), która zostanie użyta w kolejnym notatniku.

 > **Wskazówka:**  
 > Aby uniknąć potencjalnych problemów, najlepiej zapisać ramkę w sposób nawiązujący do tego notatnika, np. `flight_df_01`.

 Tutaj zapisz ramkę w najdogodniejszy dla siebie sposób.

In [ ]:
flight_delays_df.info()

In [ ]:
flight_delays_df.to_csv("../data/processed/flight_delays_01.csv", index=False)

In [ ]:
flight_df.info()

In [ ]:
flight_df.to_csv("../data/processed/flight_01.csv", index=False)